In [1]:
import pandas as pd
import math
from typing import List
from datetime import datetime


In [44]:
test = True

In [2]:
import sys
import os

In [3]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'models'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'scripts'))

In [4]:
from models.Person import Person, Sex, TypePerson

In [5]:
from scripts.utils import random_item, random_date, random_date_between, random_int, years_between

In [6]:
haitian_names = pd.read_csv("./inputs/haitian_names.csv", encoding="latin-1")
kobo_df = pd.read_csv("./inputs/kobo_local.csv", encoding="latin-1")

In [7]:
h_fnames = list(haitian_names['FirstName'])

In [8]:
h_mnames = list(haitian_names['MiddleName'])

In [9]:
h_lnames = list(haitian_names['LastName'])

In [10]:
def get_fullname():
    f_name = random_item(h_fnames)
    m_name = random_item(h_mnames)
    l_name = random_item(h_lnames).upper()
    name = ""
    if isinstance(m_name, float):
        name = (f_name + " " + l_name).strip()
    else:
        if m_name != 'nan':
            name = (f_name + " " + m_name + " " + l_name).strip()
        else:
            name = (f_name + " " + m_name + " " + l_name).strip()
    return name

In [11]:
def get_first_name(h_fnames, h_mnames):
    f_name = random_item(h_fnames)
    m_name = random_item(h_mnames)
    name = ""
    if isinstance(m_name, float):
        name = f_name
    else:
        if m_name != 'nan':
            name = (f_name + " " + m_name).strip()
        else:
            name = (f_name).strip()
    return name


def get_last_name(h_lnames):
    return random_item(h_lnames)

In [12]:
#print(get_fullname())

In [13]:
#kobo_df

In [14]:
#departments_with_communes = kobo_df.groupby(['Departements', 'Communes', ])['Section communale/Ville'].apply(lambda x: list(set(x))).reset_index()
# Convert to an array (list of dictionaries)
#result = departments_with_communes.to_dict(orient='records')

In [15]:
#communes_with_sections = []
#for depart in result:
#    communes_with_sections.append(
#        {'name': depart["Communes"], "Department": depart['Departements'], "sections": depart['Section communale/Ville'] }
#    )

In [16]:
#len(communes_with_sections )

In [17]:
#departments_with_communes = kobo_df.groupby(['Departements', 'Communes', 'Section communale/Ville'])['Communauté/Addresse'].apply(lambda x: list(set(x))).reset_index()
#result2 = departments_with_communes.to_dict(orient='records')

In [18]:
#kobo_df

In [19]:
#communes_with_sections_locals = []
#for depart in result2:
#    communes_with_sections_locals.append(
#        {'name': depart["Communes"], "Department": depart['Departements'], "sections": depart['Section communale/Ville'], 'communities':depart['Communauté/Addresse'] }
#    )

In [20]:
#communes_with_sections_locals

In [21]:
#kobo_df

In [22]:
departments_with_structure = (
    kobo_df.groupby(['Departements', 'Communes', 'Section communale/Ville', 'Communauté/Addresse'])['Coordonnée GPS']
    .first()
    .reset_index()
    .groupby(['Departements', 'Communes', 'Section communale/Ville'])
    .apply(lambda x: x[['Communauté/Addresse', 'Coordonnée GPS']].to_dict(orient='records'), include_groups=False)
    .reset_index(name='Communauté/Addresse')
    .groupby(['Departements', 'Communes'])
    .apply(lambda x: x[['Section communale/Ville', 'Communauté/Addresse']].to_dict(orient='records'), include_groups=False)
    .reset_index(name='Section communale/Ville')
    .groupby('Departements')
    .apply(lambda x: x[['Communes', 'Section communale/Ville']].to_dict(orient='records'), include_groups=False)
    .reset_index(name='Communes')
)
result4 = departments_with_structure.to_dict(orient='records')

In [23]:
#result4[0]

In [24]:
def get_department(data: list) -> dict:
    return random_item(data)

def get_commune(data: list) -> dict:
    return random_item(data)

def get_section(data: list) -> dict:
    return random_item(data)

def get_community(data: list) -> dict:
    return random_item(data)    

def get_gps_location(community: dict) -> tuple:
    gps_location = community['Coordonnée GPS'] 
    if gps_location is None:
        return (0,0) 
    if gps_location == '':
        return (0,0)
    gps_parts = gps_location.split(" ")
    return (float(gps_parts[0]), float(gps_parts[1])) 

In [25]:
birth_date_start = datetime(1940, 1, 1)
birth_date_end = datetime(2009, 1, 1)

joined_start_date = datetime(2024, 1, 1)
joined_end_date = datetime(2027, 12, 31)

#random_date = random_date_between(birth_date_start, birth_date_end)
#random_date = random_date_between(joined_start_date, joined_end_date )
#print(random_date)

In [26]:
#department = get_department(result4)

In [27]:
#commune = get_commune(department["Communes"])

In [28]:
#section = get_section(commune['Section communale/Ville'])
#section

In [29]:
#community = get_community(section['Communauté/Addresse'])

In [30]:
#get_gps_location(community)

In [45]:
if not test:
    participants = []
    sexes = [Sex.male, Sex.female]
    type_p = [TypePerson.type_1, 
              TypePerson.type_2, 
              TypePerson.type_3, 
              TypePerson.type_4, 
              TypePerson.type_5, 
              TypePerson.type_6]
    
    for i in range(30141):
        department = get_department(result4)
        commune = get_commune(department["Communes"])
        section = get_section(commune['Section communale/Ville'])
        community = get_community(section['Communauté/Addresse'])
        gps = get_gps_location(community)
        
        person: Person = Person(
               first_name = get_first_name(h_fnames, h_mnames),
               last_name = get_last_name(h_lnames),
               sex=random_item(sexes).value,
               address=community['Communauté/Addresse'],
               phone_1='33511661',
               birth_date = random_date_between(birth_date_start, birth_date_end),
               date_joined = random_date_between(joined_start_date, joined_end_date ),
               nb_goat = random_int(0, 25),
               nb_cattle= random_int(0, 10),
               nb_sheep = random_int(0, 25),
               nb_poultry = random_int(0, 100),
               gps_lat = gps[0],
               gps_lng = gps[1],
               type_person = random_item(type_p).value
        )
        participants.append(person)

In [32]:
participants
people_dicts = [person.dict() for person in participants]
df = pd.DataFrame(people_dicts)

In [33]:
#df.head()

In [34]:
today = datetime.today()

In [35]:
ages = [years_between(today.date(), d) for d in df['birth_date']]

In [36]:
df['age'] = ages

In [37]:
#df.head()

In [38]:
age_group = ['15-29' if age < 30 else '30+' for age in df['age']]

In [39]:
df['age_group'] = age_group

In [43]:
df.groupby('age_group').count()

,first_name,last_name,address,sex,birth_date,birth_year,age,type_person,type_id,cin,...,email,phone_1,phone_2,nb_goat,nb_cattle,nb_sheep,nb_poultry,gps_lat,gps_lng,date_joined
age_group,,,,,,,,,,,,,,,,,,,,,
15-29,6259,6259,6259,6259,6259,0,6259,6259,0,0,...,0,6259,0,6259,6259,6259,6259,6259,6259,6259
30+,23882,23882,23882,23882,23882,0,23882,23882,0,0,...,0,23882,0,23882,23882,23882,23882,23882,23882,23882


In [42]:
#df.to_csv('./outputs/participants.csv')